In [1]:
import pandas as pd
import numpy as np
df1 = pd.read_csv("../datasets/startup_funding_1.csv")
# df1 = pd.read_csv("../datasets/startup_funding_2.csv")
# df1 = pd.read_csv("../datasets/startup_investments/funds.csv")

print("Information of total number of non-empty columns")
print("-------------------------------------------------")
# print(df1.info(show_counts=True))

df1.info(show_counts=True)



Information of total number of non-empty columns
-------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3044 entries, 0 to 3043
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Sr No              3044 non-null   int64 
 1   Date dd/mm/yyyy    3044 non-null   object
 2   Startup Name       3044 non-null   object
 3   Industry Vertical  2873 non-null   object
 4   SubVertical        2108 non-null   object
 5   City  Location     2864 non-null   object
 6   Investors Name     3020 non-null   object
 7   InvestmentnType    3040 non-null   object
 8   Amount in USD      2080 non-null   object
 9   Remarks            419 non-null    object
dtypes: int64(1), object(9)
memory usage: 237.9+ KB


In [2]:
print("Frequency count of missing values")
df1.apply(lambda X:sum(X.isnull())) 

Frequency count of missing values


Sr No                   0
Date dd/mm/yyyy         0
Startup Name            0
Industry Vertical     171
SubVertical           936
City  Location        180
Investors Name         24
InvestmentnType         4
Amount in USD         964
Remarks              2625
dtype: int64

In [3]:
# Copying the df
df = df1.copy()
# df = df.dropna(axis=1)
df.rename(columns = {'Sr No': 's_no', 'Date dd/mm/yyyy':'date', 'Startup Name': 'startup_name', 'Industry Vertical': 'vertical', 'SubVertical': 'subvertical', 'City  Location': 'location', 'Investors Name': 'investors_name', 'InvestmentnType': 'investment_type', 'Amount in USD': 'amount_in_usd' }, inplace = True)
df = df.drop(labels = ["s_no","Remarks", "date"], axis=1)
# df = df.drop(labels = "amount_in_usd", axis=0)
df = df[df['amount_in_usd'].notna()]
delete_row1 = df[df["amount_in_usd"]=="undisclosed"].index
delete_row2 = df[df["amount_in_usd"]=="Undisclosed"].index
delete_row3 = df[df["amount_in_usd"]=="unknown"].index
df = df.drop(delete_row1)
df = df.drop(delete_row2)
df = df.drop(delete_row3)
df.head()

print("Information of total number of non-empty columns")
print("-------------------------------------------------")
print(df.info(show_counts=True))


Information of total number of non-empty columns
-------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2073 entries, 0 to 3043
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   startup_name     2073 non-null   object
 1   vertical         1942 non-null   object
 2   subvertical      1419 non-null   object
 3   location         1938 non-null   object
 4   investors_name   2053 non-null   object
 5   investment_type  2070 non-null   object
 6   amount_in_usd    2073 non-null   object
dtypes: object(7)
memory usage: 129.6+ KB
None


In [4]:
print("Frequency count of missing values")
df.apply(lambda X:sum(X.isnull())) 


Frequency count of missing values


startup_name         0
vertical           131
subvertical        654
location           135
investors_name      20
investment_type      3
amount_in_usd        0
dtype: int64

In [5]:
#converting date to pd.datetime
# 05/072018
df['date'] = pd.to_datetime(df['date'],  errors = 'coerce', format="%d-%m-%Y")
df.head()
# df["date"].head(100)

KeyError: 'date'

In [22]:
### trying decision tree

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
import matplotlib.pyplot as plt
import re
def sanitize_amount(amount):
    # if amount.lower() == 'undisclosed' or amount == 'unknown':
    #     return 0.0
    return float(amount.replace(",", ""))

def correct_location(loc):
    if("/" in str(loc)):
        return str(loc).split("/")[0]
    return str(loc).replace("Bengaluru", "Bangalore")
    # return loc
    

def to_string(x):
    return str(x)
        
data = df.copy()
data["location"] = data["location"].apply(correct_location)
data.vertical = data.vertical.apply(to_string)
data.subvertical = data.subvertical.apply(to_string)
values = ["NaN", "nan", "Nan"]
data = data[(data.vertical != "nan") & (data.subvertical != "nan") & (data.location != "nan") & (data.investment_type.isin(values) == False)]

data.amount_in_usd = data.amount_in_usd.apply(sanitize_amount)
data.location.unique()
# data.location.unique()
data.dropna(subset=['investment_type'], inplace=True)
data.investment_type.isnull().sum()
data.to_csv("../datasets/dataset_final.csv", index=False)
